In [ ]:
!pip install chroma -q
!pip install --upgrade langchain -q
!pip install openai==0.28 -q
!pip install tiktoken -q
!pip install cohere -q
!pip install chromadb -q
!pip install -U sentence-transformers -q
!pip install langchain==0.0.340 -q
!pip install llama-index==0.8.36 -q
!pip install PyPDF2 -q
!pip install faiss-cpu -q
!pip install --upgrade fastapi -q
!pip install typing_extensions -q
!pip install langchain -q
!pip install llama-index -q
!pip install camelot-py -q
!pip install ghostscript -q
!pip install pypdf -q
!pip install tabula-py -q
!sudo apt-get install ghostscript -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 527.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
!pip show llama_index

Name: llama-index
Version: 0.8.36
Summary: Interface between LLMs and your data
Home-page: https://github.com/jerryjliu/llama_index
Author: Jerry Liu
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: beautifulsoup4, dataclasses-json, fsspec, langchain, nest-asyncio, nltk, numpy, openai, pandas, sqlalchemy, tenacity, tiktoken, typing-extensions, typing-inspect, urllib3
Required-by: 


In [ ]:
from llama_index import GPTVectorStoreIndex

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader
import os
import logging
import uuid
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import uuid
import os
import logging
from langchain.prompts import PromptTemplate
import PyPDF2
from langchain.chains import ConversationChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
#from llama_index.schema import Document
from langchain.schema.document import Document
import openai
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from tabula.io import read_pdf
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import SimpleSequentialChain
from io import BytesIO
import camelot
import tabula
import pandas as pd
from langchain.embeddings import SentenceTransformerEmbeddings
import openpyxl
from langchain.chains import  RetrievalQA



user_index_file = 'vector/chroma'
user_path = "/vector/"
os.environ["OPENAI_API_KEY"] =  "replace it with your key"
embeddings  =   OpenAIEmbeddings()
llm = OpenAI(temperature=0.6,model="gpt-3.5-turbo-instruct-0914")
memory = ConversationBufferMemory()
conversation_chain = ConversationChain(llm=llm,memory = memory)
import re

def df_col_adjust(df):
  for i in range(len(df.columns)):
    if not df.columns[i].strip():  # Check if the column name is empty
        df.rename(columns={df.columns[i]: df.columns[i-1]}, inplace=True)
  while df.iloc[0].str.strip().eq('').any() or df.iloc[0].isnull().any():
    df.columns = [col if pd.isna(df.iloc[0][i]) else f'{col}-{df.iloc[0][i]}' for i, col in enumerate(df.columns)]
    df = df.iloc[1:]
  return df

def df_to_text(df):
        df = df_col_adjust(df)
        # Initialize an empty list to store the formatted text lines
        text_lines = []

        # Iterate through the DataFrame rows
        for index, row in df.iterrows():
            # Format each record with its respective heading
            record_text = ""
            for column, value in row.items():
              column = re.sub(r'\n+', ' - ', column)
              column = re.sub(r'\s+', ' ', column)
              value = value.replace('\n', ' ')
              value = re.sub(r'\s+', ' ', value)
              value = re.sub(r'\n+', ' - ', value)
              value = value.strip()
              record_text += f"{column}: {value}"
              record_text += "\n"

            text_lines.append(record_text)

        # Combine the formatted lines into a single text string
        formatted_text = "\n".join(text_lines)

        return formatted_text

def check(df):
  from tabulate import tabulate
  return tabulate(df, headers = 'keys', tablefmt = 'psql')

# Now, text_lines contains the formatted rows with both column names and values split by '\n'



def extract_table_from_pdf(pdf_path,page_number):
    try:
      tables=camelot.read_pdf(pdf_path,pages=str(page_number))
      print(f"Extracting tables from page {page_number}")

      print("-------tables---")
      print(tables)
      print("----------")
      # Check if any tables were extracted and display them
      if len(tables) != 0:
        text = ""
        for table in tables:
          try:
            df = table.df
            print("--------------1-----------------")

            is_header = not any(str(cell).isdigit() for cell in df.columns)
            if not is_header:
              df = df.rename(columns=df.iloc[0]).drop(df.index[0])
              print("--------------2-----------------")
            text += df_to_text(df)
            print("Extracted Table:")
            print(df)
            embeddings = embedding_function.encode(df_to_text(df))
            print("Embeddings:")
            print(embeddings)

          except Exception as e:
            print(f"An error occurred: {str(e)}")
            continue
        return text
      return ''
    except Exception as e:
        # print(df)
        # Handle exceptions and return an empty string
        print(f"An error occurred: {str(e)}")
        return ''
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        file_bytes=file.read()
        text = ''
        for page_number, page in enumerate(reader.pages, start=1):
          text += "\n PAGE_START \n"
          text += page.extract_text()

          table_text=extract_table_from_pdf(pdf_path,page_number)
          print(table_text)
          text += "\n---TABLE DATA---\n"
          text += table_text
          print("-------------------------------")
          text += "\n---table_data_end-----\n"

          text += "\n PAGE_END \n"
    return text

# def extract_text_from_pdf(pdf_path):
#     with open(pdf_path, 'rb') as file:
#         reader = PyPDF2.PdfReader(file)
#         file_bytes=file.read()
#         text = ''
#         for page_number, page in enumerate(reader.pages, start=1):
#           # text += "\n PAGE_START \n"
#           text += page.extract_text()

#           table_text=extract_table_from_pdf(pdf_path,page_number)
#           # print(table_text)
#           # text += "\n---TABLE DATA---\n"
#           if table_text != '':
#             text+="\n-----find info for above text--------\n"
#             text += table_text
#           print("-------------------------------")
#           text += "\n----\n"

#           text += "\n \n\n \n"
#     return text



# def split_text_into_chunks(text, chunk_size,separator='\n'):
#     text_splitter = CharacterTextSplitter(
#         separator=separator,
#         chunk_size=chunk_size,
#         chunk_overlap=0
#     )
#     chunks = text_splitter.split_text(text)
#     # for i, chunk in enumerate(chunks):
#     #     print(f"Chunk {i + 1}:")
#     #     print(chunk)
#     # print(chunks)
#     # num_chunks = len(chunks)
#     return chunks

# def split_text_into_chunks(text, chunk_size,separator='\n'):
#     text_splitter = RecursiveCharacterTextSplitter(
#         separator=separator,
#         chunk_size=chunk_size,
#         chunk_overlap=0
#     )
#     chunks = text_splitter.split_text(text)
#     print(chunks)

def split_text_into_chunks(text, chunk_size=3000, chunk_overlap=300, separator='\n'):
    text_splitter = CharacterTextSplitter(
        separator=separator,
        is_separator_regex=False,  # Change this if needed based on the library's requirements
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_text(text)
    return chunks

def process_text_chunks(text_chunks):
    for chunk_index, chunk_text in enumerate(text_chunks, start=1):
        # Process each chunk as needed
        print(chunk_text)

def get_text_chunks(text):
    separator = '\n\n'
    separator = ""
    # Split text into chunks with specified parameters
    chunks = split_text_into_chunks(text, chunk_size=3000, chunk_overlap=300, separator=separator)
    return chunks

def get_documents_from_text(text_chunks, file_name, file_id):
    docs = []
    for chunk_index, chunk_text in enumerate(text_chunks, start=1):
        doc = Document(page_content=chunk_text, metadata={"source": file_name, "file_id": f"{file_id}_chunk_{chunk_index}"})
        docs.append(doc)
    return docs
# def get_documents_from_text( input_text, file_name, file_id):
#     documents = SimpleDirectoryReader(file_id).load_data()
#     docs = []
#     for doc in documents:
#         docs.append(Document(page_content=doc.text, metadata={"source": file_name,  "file_id": file_id}))
#     # for doc in documents:
#     #     print(doc)
#     #     doc.metadata={"source": file_name, "data_source_id": data_source_id, "file_id": file_id}
#     return docs


def create_vector_index(input_text, file_name, file_id):
    chunks = split_text_into_chunks(input_text, chunk_size=3000, chunk_overlap=300, separator='\n')
    documents = get_documents_from_text(chunks, file_name, file_id)

    # Create the open-source embedding function
    embedding_model_name = "gtr-t5-large"
    embedding_function = SentenceTransformerEmbeddings(model_name=embedding_model_name)

    index_file = user_index_file

    vectordb = Chroma.from_documents(
        documents=documents,
        embedding=embedding_function,
        persist_directory=index_file
    )
    return vectordb


In [ ]:
def create_retriver():
  vector = create_vector_index(input_text, file_name, file_id)
  retriever = vector.as_retriever(search_kwargs={"k":5})
  return retriever
def create_qa_chain(prompt, retriever):

  llm = OpenAI(temperature=0.6,model="gpt-3.5-turbo-instruct-0914" )

  qa_chain = RetrievalQA.from_chain_type(
    llm= llm,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True # for source code it sholud be true, if don't need False
  )
  response = qa_chain(prompt)
  return response

In [ ]:
def extract_xlsx_text(xlsx_path):
    xlsx_text = ""
    # Load workbook directly from file path
    wb = openpyxl.load_workbook(xlsx_path)

    for sheet in wb.worksheets:
        for row in sheet.iter_rows():
            # Get values in the row, handling None values
            row_values = [str(cell.value) if cell.value is not None else '' for cell in row]

            # Include rows with at least one non-empty cell
            if any(cell_value.strip() for cell_value in row_values):
                xlsx_text += ', '.join(row_values) + '\n'

    return xlsx_text

In [ ]:
pdf_path ="/content/pdf/Supplementary KYC (1).pdf"
# Extract text from the PDF file
input_text = extract_text_from_pdf(pdf_path)
print(input_text)

# Define your data_source_id, file_id, and index_id as a list
file_id = '/content/pdf'

# Define your file_name based on the input file
file_name = os.path.basename(pdf_path)
retriever = create_retriver()

In [ ]:
prompt =" which DEPTID has the least bouns? "
response = create_qa_chain(prompt, retriever)
llm_response = response
print(llm_response)

In [ ]:
def process_llm_response(llm_response):
  print(llm_response['result'])
  print('\n\nSources:')
  for source in llm_response["source_documents"]:
    print(source.metadata['source'])

In [ ]:
prompt_template  ="which DEPTID has the least bouns? "
response = create_qa_chain(prompt, retriever)
llm_response = response
process_llm_response(llm_response)